In [93]:
import mysql.connector as sql
import pandas as pd
import datetime
import uuid


In [94]:
def convert_date_col(date_str):
    date_str = date_str[0:16]
    try:
        return datetime.datetime.strptime(date_str, '%m/%d/%Y %H:%M').strftime('%Y-%m-%dT%H:%M')
    except:
        return datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M').strftime('%Y-%m-%dT%H:%M')

In [95]:
db_connection = sql.connect(host='localhost', database='benedict', user='root', password='artema2')

In [96]:
qry = "SELECT fin, event_date  as eventDate, "
qry = qry + "event_name AS eventName, result_val as resultValue, "
qry = qry + "result_unit AS resultUnit, event_category AS eventCategory "
qry = qry + "FROM benedict.sepsis_case_events ORDER BY fin, event_date "
# qry = qry + " LIMIT 1000;"
df = pd.read_sql(qry, con=db_connection)

In [97]:
df.head()

,fin,eventDate,eventName,resultValue,resultUnit,eventCategory
0,+/wwoxFTiWTn,12/18/2013 12:57,Temperature Conversion,37.1,DegC,vitals
1,+/wwoxFTiWTn,12/18/2013 12:57,Respiratory Rate,18.0,RPM,vitals
2,+/wwoxFTiWTn,12/18/2013 12:57,Diastolic BP,99.0,mmHg,vitals
3,+/wwoxFTiWTn,12/18/2013 12:57,Pulse,72.0,BPM,vitals
4,+/wwoxFTiWTn,12/18/2013 12:57,Systolic BP,164.0,mmHg,vitals


In [98]:
df.shape

(6047543, 6)

In [99]:
df['eventDate'] = df['eventDate'].apply(lambda x: convert_date_col(x))

In [100]:
df.head()

,fin,eventDate,eventName,resultValue,resultUnit,eventCategory
0,+/wwoxFTiWTn,2013-12-18T12:57,Temperature Conversion,37.1,DegC,vitals
1,+/wwoxFTiWTn,2013-12-18T12:57,Respiratory Rate,18.0,RPM,vitals
2,+/wwoxFTiWTn,2013-12-18T12:57,Diastolic BP,99.0,mmHg,vitals
3,+/wwoxFTiWTn,2013-12-18T12:57,Pulse,72.0,BPM,vitals
4,+/wwoxFTiWTn,2013-12-18T12:57,Systolic BP,164.0,mmHg,vitals


In [101]:
df_pat = df['fin'].value_counts().rename_axis('fin').reset_index(name='counts')
df_pat['patientId'] = [uuid.uuid4() for _ in range(len(df_pat.index))]
df_pat.head()

,fin,counts,patientId
0,aePTw3ZLfusq,22060,c096f95c-2546-4806-931b-3e75eb6dce31
1,zjI+728vMb6P,20587,b0836bdf-21e2-4888-9dc5-bfcc98aca747
2,E0v8kM7hynUI,19497,aca8f6e0-837b-467b-9a3c-ede3cfacadec
3,bjS47Ql19Qi0,19245,bc107d6a-7ca8-4d8c-a19c-edbb5d18350c
4,WtR0GYU74gOB,19172,02806116-784d-4eaa-8f96-336be15a8598


In [102]:
df_events = pd.merge(df, df_pat, on='fin', how='inner')
df_events.head()

,fin,eventDate,eventName,resultValue,resultUnit,eventCategory,counts,patientId
0,+/wwoxFTiWTn,2013-12-18T12:57,Temperature Conversion,37.1,DegC,vitals,112,941822f2-f9f7-4bfe-b956-b8a610e448cd
1,+/wwoxFTiWTn,2013-12-18T12:57,Respiratory Rate,18.0,RPM,vitals,112,941822f2-f9f7-4bfe-b956-b8a610e448cd
2,+/wwoxFTiWTn,2013-12-18T12:57,Diastolic BP,99.0,mmHg,vitals,112,941822f2-f9f7-4bfe-b956-b8a610e448cd
3,+/wwoxFTiWTn,2013-12-18T12:57,Pulse,72.0,BPM,vitals,112,941822f2-f9f7-4bfe-b956-b8a610e448cd
4,+/wwoxFTiWTn,2013-12-18T12:57,Systolic BP,164.0,mmHg,vitals,112,941822f2-f9f7-4bfe-b956-b8a610e448cd


In [103]:
df_events = df_events[['patientId', 'eventDate', 'eventName', 'resultValue', 'resultUnit', 'eventCategory']]

In [104]:
df_events.to_csv('sepsis_events_sorted.csv')
df_pat.to_csv('sepsis_events_patients.csv')